In [42]:
import os
from lxml import etree

# Define the path to the folder containing the SVG files
folder_path = "Site 1/svg_output"

# Namespace handling for SVG and Inkscape
namespace = {
    "svg": "http://www.w3.org/2000/svg",
    "inkscape": "http://www.inkscape.org/namespaces/inkscape"
}

# Dictionary to store labels from each floor file
all_labels = {}

# Process each SVG file in the folder
for i in range(1, 8):  # assuming 7 floors named floor_1.svg to floor_7.svg
    file_path = os.path.join(folder_path, f"floor_{i}.svg")

    # Load the SVG file
    with open(file_path, "rb") as file:
        content = file.read().decode("ISO-8859-1")

    # Remove the encoding declaration if it exists
    content = content.replace('<?xml version="1.0" encoding="ISO-8859-1"?>', '')

    # Parse from the string content without encoding declaration
    tree = etree.fromstring(content.encode("utf-8"))

    # Extract labels from the current file
    labels = []
    for element in tree.xpath("//*[@inkscape:label]", namespaces=namespace):
        label = element.get("{http://www.inkscape.org/namespaces/inkscape}label")
        labels.append(label)

    # Store extracted labels in the dictionary, converting to a set for unique labels
    all_labels[f"floor_{i}"] = set(labels)

# Print all extracted labels for each floor
print("Extracted Inkscape Labels for Each Floor:")
for floor, labels in all_labels.items():
    print(f"\n{floor}:")
    for label in sorted(labels):  # Sorted for easier comparison
        print(f" - {label}")

# Compare labels between floors
reference_labels = all_labels["floor_1"]
consistent = True

# Store mismatches for summary
mismatches = {}

for floor, labels in all_labels.items():
    if labels != reference_labels:
        # Record the mismatches
        only_in_floor = labels - reference_labels
        only_in_reference = reference_labels - labels
        mismatches[floor] = {
            "only_in_floor": only_in_floor,
            "only_in_reference": only_in_reference
        }
        consistent = False

# Display any inconsistencies in detail
if consistent:
    print("\nAll floors have consistent labels.")
else:
    print("\nInconsistencies found between floors:")
    for floor, diff in mismatches.items():
        print(f"\n{floor} differs from floor_1:")
        if diff["only_in_floor"]:
            print("  Labels only in this floor:")
            for label in sorted(diff["only_in_floor"]):
                print(f"    - {label}")
        if diff["only_in_reference"]:
            print("  Labels missing in this floor (present in floor_1):")
            for label in sorted(diff["only_in_reference"]):
                print(f"    - {label}")

Extracted Inkscape Labels for Each Floor:

floor_1:
 - 1
 - AOA Aina nakyvissa
 - AOA Ala huone
 - AOA Ala huoneisto
 - AOA Huomiomerkinnat
 - AOA Irtaimistovarastojen seinät.var. seinät
 - AOA Julkisivu- ja tasopiirustukset
 - AOA Kalusteet kiinto
 - AOA Kalusteet kiinto litterat
 - AOA Kalusteet varusteet
 - AOA Korkomerkinnät
 - AOA Lattia- ja pohjapiirustus
 - AOA Lattiakaivot
 - AOA Moduulit
 - AOA Muutosnuolet
 - AOA Palomerkinnät
 - AOA Pohjapiirustus
 - AOA Pyörähdysympyrät
 - AOA Seinaselitteet
 - AOA Sisakatto- ja pohjapiirustus
 - AOA Sisaosat kantavat
 - AOA Sisaosat kevyet
 - AOA Tontti
 - AOA Tyopiirustusmerkinnat
 - AOA Tyopiirustusmerkinnat Vss
 - AOA Työpiirustusmitat
 - AOA Urakkaraja
 - AOA merkinnat
 - AOA sisäkattoselite

floor_2:

floor_3:

floor_4:

floor_5:
 - 1
 - AOA Aina nakyvissa
 - AOA Ala huone
 - AOA Ala huoneisto
 - AOA Hormit
 - AOA Julkisivu- ja tasopiirustukset
 - AOA Kalusteet kiinto
 - AOA Kalusteet kiinto litterat
 - AOA Kalusteet varusteet
 - AOA 

In [43]:
# Keywords to identify outside walls and rooms
wall_keywords = ["seinät", "seinaselitteet"]
room_keywords = ["huone", "pohjapiirustus"]  # Floor plans may also be considered

# Lists to store identified walls and rooms
outside_walls = []
rooms = []

# Check each label and classify it
for label in all_labels["floor_1"]:
    if any(keyword in label.lower() for keyword in wall_keywords):
        outside_walls.append(label)
    if any(keyword in label.lower() for keyword in room_keywords):
        rooms.append(label)

# Print identified outside walls and rooms
print("Identified Outside Walls:")
for wall in outside_walls:
    print(f" - {wall}")

print("\nIdentified Rooms:")
for room in rooms:
    print(f" - {room}")

Identified Outside Walls:
 - AOA Irtaimistovarastojen seinät.var. seinät
 - AOA Seinaselitteet

Identified Rooms:
 - AOA Sisakatto- ja pohjapiirustus
 - AOA Pohjapiirustus
 - AOA Ala huone
 - AOA Lattia- ja pohjapiirustus
 - AOA Ala huoneisto


In [53]:
import os
from lxml import etree

# Define the path to the folder containing the SVG files
folder_path = "Site 1/svg_output"
output_folder = "Site 1/svg_output_filtered"  # Folder to save the filtered SVGs

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Namespace handling for SVG and Inkscape
namespace = {
    "svg": "http://www.w3.org/2000/svg",
    "inkscape": "http://www.inkscape.org/namespaces/inkscape"
}

# Keywords to identify outside walls and rooms
wall_keywords = ["seinät", "seinaselitteet"]
room_keywords = ["huone", "pohjapiirustus", "lattia", "sisakatto"]

# Process each SVG file in the folder
for i in range(1, 8):  # assuming 7 floors named floor_1.svg to floor_7.svg
    file_path = os.path.join(folder_path, f"floor_{i}.svg")
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Warning: {file_path} does not exist. Skipping this floor.")
        continue

    # Load the SVG file
    print(f"Processing {file_path}...")
    try:
        with open(file_path, "rb") as file:
            content = file.read().decode("ISO-8859-1")
        
        # Remove the encoding declaration if it exists
        content = content.replace('<?xml version="1.0" encoding="ISO-8859-1"?>', '')
        
        # Parse from the string content without encoding declaration
        tree = etree.fromstring(content.encode("utf-8"))
        
        # Remove all <text> elements (text components)
        for text_element in tree.xpath("//svg:text", namespaces=namespace):
            parent = text_element.getparent()
            parent.remove(text_element)
        
        # Iterate over all path elements and remove those that are not inside a folder (group)
        for path in tree.xpath("//svg:path", namespaces=namespace):
            # Check if the path has any parent group (folder)
            parent_group = path.getparent()
            
            if parent_group is None or parent_group.tag != etree.QName(namespace['svg'], 'g'):
                # If it's not inside a group (folder), remove it
                parent = path.getparent()
                parent.remove(path)
        
        # Iterate over elements and filter only the ones that match wall or room keywords
        for element in tree.xpath("//*[@inkscape:label]", namespaces=namespace):
            label = element.get("{http://www.inkscape.org/namespaces/inkscape}label")
            # Remove the element if the label does not contain any of the wall or room keywords
            if not any(keyword in label.lower() for keyword in wall_keywords + room_keywords):
                parent = element.getparent()
                parent.remove(element)
        
        # Convert the modified tree back to a string and save to a new SVG file in the output folder
        output_content = etree.tostring(tree, pretty_print=True, encoding="utf-8").decode("utf-8")
        output_file_path = os.path.join(output_folder, f"floor_{i}_filtered.svg")
        
        # Write the output content to a new file
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write('<?xml version="1.0" encoding="utf-8"?>\n' + output_content)
        
        print(f"Filtered SVG for floor {i} saved as '{output_file_path}'")
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

print("Filtering complete. Filtered SVGs have been saved.")

Processing Site 1/svg_output/floor_1.svg...
Filtered SVG for floor 1 saved as 'Site 1/svg_output_filtered/floor_1_filtered.svg'
Processing Site 1/svg_output/floor_2.svg...
Filtered SVG for floor 2 saved as 'Site 1/svg_output_filtered/floor_2_filtered.svg'
Processing Site 1/svg_output/floor_3.svg...
Filtered SVG for floor 3 saved as 'Site 1/svg_output_filtered/floor_3_filtered.svg'
Processing Site 1/svg_output/floor_4.svg...
Filtered SVG for floor 4 saved as 'Site 1/svg_output_filtered/floor_4_filtered.svg'
Processing Site 1/svg_output/floor_5.svg...
Filtered SVG for floor 5 saved as 'Site 1/svg_output_filtered/floor_5_filtered.svg'
Processing Site 1/svg_output/floor_6.svg...
Filtered SVG for floor 6 saved as 'Site 1/svg_output_filtered/floor_6_filtered.svg'
Processing Site 1/svg_output/floor_7.svg...
Filtered SVG for floor 7 saved as 'Site 1/svg_output_filtered/floor_7_filtered.svg'
Filtering complete. Filtered SVGs have been saved.


In [54]:
import os
from lxml import etree
import xml.etree.ElementTree as ET

# Define the path to the folder containing the SVG files
folder_path = "Site 1/svg_output"
output_folder = "Site 1/svg_output_filtered"  # Folder to save the filtered SVGs

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Namespace handling for SVG and Inkscape
namespace = {
    "svg": "http://www.w3.org/2000/svg",
    "inkscape": "http://www.inkscape.org/namespaces/inkscape"
}

# Keywords to identify outside walls and rooms
wall_keywords = ["seinät", "seinaselitteet"]
room_keywords = ["huone", "pohjapiirustus", "lattia", "sisakatto"]

# Function to stack filtered SVGs
def stack_floors(floor_svgs):
    svg_ns = {'svg': 'http://www.w3.org/2000/svg'}
    root = ET.Element('svg', xmlns="http://www.w3.org/2000/svg", width="500", height=str(300 * len(floor_svgs)))
    
    for idx, floor_svg in enumerate(floor_svgs):
        # Parse each floor SVG
        tree = ET.parse(floor_svg)
        floor_root = tree.getroot()
        
        # Move each floor down by a multiple of 300 (the height of each floor)
        for element in floor_root.findall('.//svg:*', svg_ns):
            if 'transform' in element.attrib:
                element.attrib['transform'] = f"translate(0,{300 * idx})"
            else:
                element.attrib['transform'] = f"translate(0,{300 * idx})"
        
        # Append to the main SVG
        for child in floor_root:
            root.append(child)
    
    # Convert back to a string or write to a file
    new_svg = ET.tostring(root, encoding='unicode')
    with open('stacked_floors.svg', 'w') as f:
        f.write(new_svg)

# Process each SVG file in the folder
filtered_svgs = []  # List to store paths to filtered SVG files
for i in range(1, 8):  # assuming 7 floors named floor_1.svg to floor_7.svg
    file_path = os.path.join(folder_path, f"floor_{i}.svg")
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Warning: {file_path} does not exist. Skipping this floor.")
        continue

    # Load the SVG file
    print(f"Processing {file_path}...")
    try:
        with open(file_path, "rb") as file:
            content = file.read().decode("ISO-8859-1")
        
        # Remove the encoding declaration if it exists
        content = content.replace('<?xml version="1.0" encoding="ISO-8859-1"?>', '')
        
        # Parse from the string content without encoding declaration
        tree = etree.fromstring(content.encode("utf-8"))
        
        # Remove all <text> elements (text components)
        for text_element in tree.xpath("//svg:text", namespaces=namespace):
            parent = text_element.getparent()
            parent.remove(text_element)
        
        # Iterate over all path elements and remove those that are not inside a folder (group)
        for path in tree.xpath("//svg:path", namespaces=namespace):
            # Check if the path has any parent group (folder)
            parent_group = path.getparent()
            
            if parent_group is None or parent_group.tag != etree.QName(namespace['svg'], 'g'):
                # If it's not inside a group (folder), remove it
                parent = path.getparent()
                parent.remove(path)
        
        # Iterate over elements and filter only the ones that match wall or room keywords
        for element in tree.xpath("//*[@inkscape:label]", namespaces=namespace):
            label = element.get("{http://www.inkscape.org/namespaces/inkscape}label")
            # Remove the element if the label does not contain any of the wall or room keywords
            if not any(keyword in label.lower() for keyword in wall_keywords + room_keywords):
                parent = element.getparent()
                parent.remove(element)
        
        # Convert the modified tree back to a string and save to a new SVG file in the output folder
        output_content = etree.tostring(tree, pretty_print=True, encoding="utf-8").decode("utf-8")
        output_file_path = os.path.join(output_folder, f"floor_{i}_filtered.svg")
        
        # Write the output content to a new file
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write('<?xml version="1.0" encoding="utf-8"?>\n' + output_content)
        
        filtered_svgs.append(output_file_path)
        print(f"Filtered SVG for floor {i} saved as '{output_file_path}'")
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Stack the filtered SVG files
print("Stacking filtered SVGs...")
stack_floors(filtered_svgs)
print("Stacking complete. Stacked SVG saved as 'stacked_floors.svg'.")

Processing Site 1/svg_output/floor_1.svg...
Filtered SVG for floor 1 saved as 'Site 1/svg_output_filtered/floor_1_filtered.svg'
Processing Site 1/svg_output/floor_2.svg...
Filtered SVG for floor 2 saved as 'Site 1/svg_output_filtered/floor_2_filtered.svg'
Processing Site 1/svg_output/floor_3.svg...
Filtered SVG for floor 3 saved as 'Site 1/svg_output_filtered/floor_3_filtered.svg'
Processing Site 1/svg_output/floor_4.svg...
Filtered SVG for floor 4 saved as 'Site 1/svg_output_filtered/floor_4_filtered.svg'
Processing Site 1/svg_output/floor_5.svg...
Filtered SVG for floor 5 saved as 'Site 1/svg_output_filtered/floor_5_filtered.svg'
Processing Site 1/svg_output/floor_6.svg...
Filtered SVG for floor 6 saved as 'Site 1/svg_output_filtered/floor_6_filtered.svg'
Processing Site 1/svg_output/floor_7.svg...
Filtered SVG for floor 7 saved as 'Site 1/svg_output_filtered/floor_7_filtered.svg'
Stacking filtered SVGs...
Stacking complete. Stacked SVG saved as 'stacked_floors.svg'.
